In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from talib import BBANDS, RSI, MA_Type, ATR, MFI
import datetime as dt

In [2]:
# Download historical data
tickerSymbol = input("What stock are we crushing today?: ")
# Get data on this ticker
tickerData = yf.Ticker(tickerSymbol)
# Get the historical prices for this ticker
df = tickerData.history(period='60d', interval='5m')

What stock are we crushing today?:  NFLX


In [3]:
# Calculate Bollinger Bands
df['upper_band'], df['middle_band'], df['lower_band'] = BBANDS(df['Close'], timeperiod=20)

# Calculate RSI
df['rsi'] = RSI(df['Close'], timeperiod=14)

# Calculate Keltner Channels
df['keltner_middle'] = df['Close'].rolling(window=20).mean()
df['atr'] = ATR(df['High'], df['Low'], df['Close'], timeperiod=20)
df['keltner_upper'] = df['keltner_middle'] + (df['atr'] * 1.5)
df['keltner_lower'] = df['keltner_middle'] - (df['atr'] * 1.5)

# Calculate Money Flow Index
df['mfi'] = MFI(df['High'], df['Low'], df['Close'], df['Volume'], timeperiod=14)

In [4]:
starting_balance = 50000
risk_amount = (.40 * starting_balance) # 40% of starting balance
stop_loss_pct = 0.15 # 15%
balance = starting_balance
position = 0
entry_price = 0
num_shares = 0

In [5]:
# Initialize trade journal
trade_journal = pd.DataFrame(columns=["Date", "Action", "Price", "Shares", "Balance"])

In [6]:
# Iterate over the data
for i, row in df.iterrows():
    # Check if RSI is below 20 and close price is below the lower Keltner Channel or lower Bollinger Band
    if row['rsi'] < 30 and (row['Close'] <  row['keltner_lower'] or row['Close'] < row['lower_band']):
        if position == 0:
            num_shares = int(risk_amount / row['Close'])
            balance -= num_shares * row['Close']
            position += num_shares
            entry_price = row['Close']
            trade_journal = trade_journal.append({"Date": row.name, "Action": "Buy", "Price": row["Close"], "Shares": num_shares, "Balance": balance}, ignore_index=True)
    elif ((row['rsi'] > 80 and (row['Close'] > row['keltner_upper'] or row['Close'] > row['upper_band'])) or 
          (entry_price > 0 and row['Close'] < entry_price * (1 - stop_loss_pct)) or 
          (row['mfi'] < 57 and row['rsi'] > 80)):
        if position > 0:
            balance += position * row['Close']
            trade_journal = trade_journal.append({"Date": row.name, "Action": "Sell", "Price": row["Close"], "Shares": position, "Balance": balance}, ignore_index=True)
            position = 0

In [7]:
# Separate dataframes for buy and sell operations
buy_df = trade_journal[trade_journal["Action"] == "Buy"].copy()
sell_df = trade_journal[trade_journal["Action"] == "Sell"].copy()


In [8]:
# Ensure that the number of 'Buy' transactions is equal to the number of 'Sell' transactions
# A single 'Buy' without a corresponding 'Sell' is not counted as a complete trade
if len(buy_df) != len(sell_df):
    if len(buy_df) > len(sell_df):
        buy_df = buy_df.iloc[:-1, :]  # Remove the last 'Buy' operation
    else:
        sell_df = sell_df.iloc[:-1, :]  # Remove the last 'Sell' operation

In [9]:
# Compute the number of completed trades
num_of_trades = len(buy_df)

In [10]:
# Reset indices of buy_df and sell_df
buy_df.reset_index(drop=True, inplace=True)
sell_df.reset_index(drop=True, inplace=True)

In [11]:
# Compute Win/Loss ratio
sell_df["Win"] = sell_df["Price"] > buy_df["Price"]
win_loss_ratio = sell_df["Win"].sum() / num_of_trades

In [12]:
# Compute ROI
sell_df["ROI"] = (sell_df["Price"] - buy_df["Price"]) / buy_df["Price"]
average_roi = sell_df["ROI"].mean()

In [13]:
# Aggregate the results
result = {"Ticker": tickerSymbol,
          "Average ROI": average_roi * 100}

final_df = pd.DataFrame(result, index=[0])

In [14]:
# Compute the win ratio and the total profit
win_ratio = sell_df["Win"].sum() / num_of_trades
total_profit = balance - starting_balance

In [15]:
# Compute profit percentage
profit_percentage = (total_profit / starting_balance) * 100


In [16]:
# Metrics
print(final_df)
print(f"Trade Journal:\n{trade_journal}")
print(f"Final Balance: ${balance:.2f}")
print(f"Profit: ${total_profit:.2f}")
print(f"Number of Trades: {num_of_trades}")
print(f"Win Rate Percentage: {win_ratio * 100:.2f}%")
print("Profit Percentage: ", profit_percentage, "%")


  Ticker  Average ROI
0   NFLX     3.952659
Trade Journal:
                        Date Action       Price Shares       Balance
0  2023-03-17 14:40:00-04:00    Buy  300.700012     66  30153.799194
1  2023-03-30 09:30:00-04:00   Sell  339.779999     66  52579.279114
2  2023-04-05 09:45:00-04:00    Buy  341.287811     58  32784.586060
3  2023-04-11 09:30:00-04:00   Sell  346.213196     58  52864.951416
4  2023-04-11 15:40:00-04:00    Buy  338.380005     59  32900.531128
5  2023-04-24 09:35:00-04:00   Sell  331.779999     59  52475.551056
6  2023-04-25 09:45:00-04:00    Buy  324.744995     61  32666.106354
7  2023-05-08 09:45:00-04:00   Sell  327.024994     61  52614.630981
8  2023-05-09 14:25:00-04:00    Buy  332.154999     60  32685.331055
9  2023-05-18 09:30:00-04:00   Sell  352.670013     60  53845.531860
10 2023-05-19 09:35:00-04:00    Buy  365.779999     54  34093.411926
11 2023-05-26 12:00:00-04:00   Sell  372.500000     54  54208.411926
12 2023-06-02 09:55:00-04:00    Buy  397.829

In [17]:
# Print statements based on profit percentage
if trade_journal.iloc[-1]['Action'] == 'Buy':
    print("Looks bad, but you still have a position open.")
elif profit_percentage > 2:
    print("Looks like we're crushin' it..$$$")
elif 0 <= profit_percentage <= 1:
    print("Could be better..")
else:
    print("Ouch.. back to the drawing board for this one..")

Looks like we're crushin' it..$$$
